In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
import pandas as pd
import matplotlib.pylab as plt
#import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/homebrew/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
submit=pd.read_csv('submit1.csv', sep=',')

In [3]:
def get_sample(index, frame): #Получение временного ряда по индексу
    uuid = frame.iloc[index, 0]
    with open(f'data/{uuid}', 'r') as f:
        content = f.read()
    rows = [line.split('\t') for line in content.strip().split('\n')]
    data = pd.DataFrame(rows, columns=['time', 'delta_p', 'p_'], dtype=float)
    return data

In [4]:
def find_empty_indexes_2(data):
    """
    Поиск id файлов из строк датафрейма для их исключения на этапе удаления данных из датафрейма
    """
    B=[]
    for i in range(data.shape[0]):
        try:
            df=get_sample(i,data)
        except ValueError:
            B.append(data.iloc[i,0])
    return B

In [5]:
def plot_save_equal(data, uuid, direct):
    """
Сохранение графика рядов с равными ценами делений по uuid
data: выход функции get_sample
uuid: id файла с данными
direct: папка, в которую идёт сохранение (внутри папки imgs)
    """
    figure, ax = plt.subplots(figsize=(7, 7), dpi=32)
    ax.loglog(data['time'], data['delta_p'], 'o', label='Series 1')
    ax.loglog(data['time'], data['p_'], 'o', label='Series 2')
    ax.set_aspect('equal')
    plt.axis('off')
    plt.savefig(f'imgs_equal_test/{direct}/{uuid}.jpg')
    plt.close()

In [6]:
def save_plots(data, direct, equal=False):
    """
Сохранение графиков ряда из куска датасета в директорию
    """
    empty_indexes=find_empty_indexes(data)#Получение индексов массивов с id файлов, которых нет
    print('Были успешно найдены индексы с отсутствующими файлами!')
    for i in range(data.shape[0]):
        if i not in empty_indexes:#Проверка на отсутствие в списке с пустыми файлами
            df=get_sample(i,data)
            if equal==True:
                plot_save_equal(df, data.iloc[i,0], direct)
            else:
                plot_save(df, data.iloc[i,0], direct)
            plt.close()
        if i%1000==0:
            print(f'Загружено {i} изображений!')

In [7]:
model_class=models.resnet50(pretrained=False)
model_class.fc = nn.Linear(model_class.fc.in_features, 8, bias=True)
model_class.load_state_dict(torch.load('finetuned_resnet50_new.pth', weights_only=True, map_location=torch.device('cpu')))

/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


<All keys matched successfully>

In [8]:
class ImageDataset_2(Dataset):
    """
    Класс с торча, возвращает датасет из картинок и таргетов
    Принимает на вход директорию и датафрейм из предыдущих пунктов
    """
    def __init__(self, image_dir, data, transform=None):
        self.image_dir = image_dir
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.data.iloc[idx, 0])  
        image = Image.open(f'{img_name}.jpg').convert("RGB")
        if self.transform:
            image = self.transform(image)
        target = torch.tensor(self.data.iloc[idx, 1:].values.astype('float32'), dtype=torch.float32)
        file=self.data.iloc[idx, 0]
        #target = torch.tensor(self.data.iloc[idx, 1:].values.astype('int'), dtype=torch.float32)
        return image, target, file

In [9]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [10]:
def prepare_data_2(data):
    """
    Подготовка датафрейма с таргетами
    Возвращает подготовленный датафрейм
    """
    B=find_empty_indexes_2(data)#Поиск id с отсутствующими файлами для исключения
    print(B)
    df=data[~data['file_name'].isin(B)]#Фильтрация данных
    #Удаление всех столбцов, кроме id и таргетов
    df=df.drop(['Влияние ствола скважины_details', 'Радиальный режим_details', 'Линейный режим_details','Билинейный режим_details','Сферический режим_details','Граница постоянного давления_details','Граница непроницаемый разлом_details'],axis=1)
    return df

In [11]:
submit_class=prepare_data_2(submit)

['846291a2-6475-47ac-8f10-08eac6b93fcb']


In [12]:
testset_equal=ImageDataset_2("imgs_equal_test/test", submit_class, transform=transform)
testloader = DataLoader(testset_equal, batch_size=499, shuffle=False)#Получение итерируемого объекта с тренировочным датасетом

In [13]:
model_class.eval()
with torch.no_grad():
    for images, labels, files in testloader:
        #print(labels.shape)
        #images, labels = images.to(device), labels.to(device)
        outputs = model_class(images)
        file=np.array(files)

In [14]:
outputs=torch.sigmoid(outputs)
predicted = (outputs > 0.7).int()

In [15]:
predicted

tensor([[0, 1, 1,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 1, 0, 1],
        [0, 1, 0,  ..., 0, 0, 0],
        ...,
        [0, 1, 1,  ..., 0, 1, 0],
        [0, 1, 1,  ..., 0, 0, 1],
        [0, 1, 0,  ..., 0, 1, 0]], dtype=torch.int32)

In [16]:
p=predicted.cpu().detach().numpy()
df_classification=pd.DataFrame(p)

In [17]:
df_f=pd.DataFrame(file)

In [18]:
df_itog=pd.concat([df_f, df_classification], axis=1)

In [19]:
df_itog

,0,0,1,2,3,4,5,6,7
0,283bbcbc-ee90-4a6a-ae06-8392104a2c10,0,1,1,1,1,0,0,0
1,cabe112d-93ae-4bb2-9585-a36d3bc31349,0,0,1,0,0,1,0,1
2,3eaf5015-fe2c-4f54-87c7-cd91142e19f7,0,1,0,0,0,0,0,0
3,612e4ad5-98a9-4a53-b034-d14d02305126,0,1,0,0,0,0,1,0
4,5bf5e5d7-5ad2-49ba-8bcc-719831ad60bc,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
494,6fe55825-442f-4a73-a659-50346c8096d2,0,1,1,0,0,1,1,0
495,10f9be1f-9ccb-4fa2-ad74-69d53941a6a4,0,1,1,0,0,1,0,0
496,903787df-01b6-4308-b037-11c7ba81eb4e,0,1,1,0,0,0,1,0
497,3c264f8a-df09-4beb-b64f-879c515e88b0,0,1,1,0,0,0,0,1


In [20]:
df_itog.columns=submit.columns[:9]

In [21]:
df_itog

,file_name,Некачественное ГДИС,Влияние ствола скважины,Радиальный режим,Линейный режим,Билинейный режим,Сферический режим,Граница постоянного давления,Граница непроницаемый разлом
0,283bbcbc-ee90-4a6a-ae06-8392104a2c10,0,1,1,1,1,0,0,0
1,cabe112d-93ae-4bb2-9585-a36d3bc31349,0,0,1,0,0,1,0,1
2,3eaf5015-fe2c-4f54-87c7-cd91142e19f7,0,1,0,0,0,0,0,0
3,612e4ad5-98a9-4a53-b034-d14d02305126,0,1,0,0,0,0,1,0
4,5bf5e5d7-5ad2-49ba-8bcc-719831ad60bc,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
494,6fe55825-442f-4a73-a659-50346c8096d2,0,1,1,0,0,1,1,0
495,10f9be1f-9ccb-4fa2-ad74-69d53941a6a4,0,1,1,0,0,1,0,0
496,903787df-01b6-4308-b037-11c7ba81eb4e,0,1,1,0,0,0,1,0
497,3c264f8a-df09-4beb-b64f-879c515e88b0,0,1,1,0,0,0,0,1


In [22]:
p.shape

(499, 8)